In [149]:
%pip install tensorflow
%pip install keras
%pip install Keras-Preprocessing
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [150]:
import tensorflow as tf
print("Versão do TensorFlow:", tf.__version__)
import keras as K
print("Versão do Keras:", K.__version__)


Versão do TensorFlow: 2.16.1
Versão do Keras: 3.3.3


In [151]:
# Imports
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [152]:
# Inicializando a Rede Neural Convolucional
def build_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model
classifier = Sequential()

In [153]:
# Passo 1 - Primeira Camada de Convolução
Conv2D(32, (3, 3), activation='relu', input_shape= (28,28,1))

c:\Users\T-GAMER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


<Conv2D name=conv2d_35, built=False>

In [154]:
# Passo 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [155]:
# Adicionando a Segunda Camada de Convolução
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))

In [156]:
classifier.add(MaxPooling2D(pool_size = (2, 2)))


In [157]:
# Passo 3 - Flattening
classifier.add(Flatten())

In [158]:
# Passo 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [159]:
# Compilando a rede
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [160]:
# # Criando os objetos train_datagen e validation_datagen com as regras de pré-processamento das imagens
# import keras.preprocessing.image as kp

# train_datagen = kp.ImageDataGenerator(rescale = 1./255,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True)

# validation_datagen = kp.ImageDataGenerator(rescale = 1./255)
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define the input directory
input_dir = 'dataset_personagens\\training_set'

# Parameters
img_size = 200  # Example size for MNIST (28x28)
categories = ['bart', 'homer']  # Replace with your actual categories

# Function to load and preprocess images
def load_images(input_dir, img_size, categories):
    images = []
    labels = []
    
    for category in categories:
        category_path = os.path.join(input_dir, category)
        class_num = categories.index(category)
        
        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            if os.path.isfile(img_path) and img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                # Load image
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    continue
                # Resize image
                img = cv2.resize(img, (img_size, img_size))
                # Normalize image
                img = img / 255.0
                # Append to list
                images.append(img)
                labels.append(class_num)
    
    return np.array(images), np.array(labels)

# Load and preprocess images
images, labels = load_images(input_dir, img_size, categories)

# Reshape images for CNN
images = images.reshape(-1, img_size, img_size, 1)

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

print("Data preprocessed successfully.")


Data preprocessed successfully.


In [161]:
X_train

array([[[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]],


       [[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
 

In [162]:
y_train

array([0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 0])

In [163]:
labels
input_shape = (img_size, img_size, 1)
model = build_cnn(input_shape, 2)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [164]:
# # Pré-processamento das imagens de treino e validação
# training_set = train_datagen.flow_from_directory('dataset_treino',
#                                                  target_size = (64, 64),
#                                                  batch_size = 32,
#                                                  class_mode = 'binary')

# validation_set = validation_datagen.flow_from_directory('dataset_validation',
#                                                         target_size = (64, 64),
#                                                         batch_size = 32,
#                                                         class_mode = 'binary')

In [165]:
# Executando o treinamento (esse processo pode levar bastante tempo, dependendo do seu computador)
model.fit(X_train,
                         steps_per_epoch = 8000,
                         epochs = 5,
                         validation_data = y_train,
                         validation_steps = 2000)

Epoch 1/5


ValueError: None values not supported.